# Classification de sites web

In [ ]:
# Importation des librairies
import pandas            as pd
import numpy             as np
import matplotlib.pyplot as plt
import string            as str
import xgboost           as xgb
import torch.nn          as nn
import torch
import re
import nltk

In [ ]:
from nltk.corpus import stopwords # Les stopwords sont des mots qui n'apportent pas de sens à la phrase
from nltk.stem   import PorterStemmer # Le stemmer permet de réduire les mots à leur racine
from sklearn.feature_extraction.text import CountVectorizer # Permet de transformer le texte en vecteur (bag of words)

In [ ]:
nltk.download('stopwords')
nltk.download('punkt')

# Lecture des données

In [ ]:
df_train = pd.read_csv('../data/train.csv')

In [ ]:
df_test = pd.read_csv('../data/test.csv')

In [ ]:
df_train.head()

In [ ]:
df_test.head()

In [ ]:
# On affiche les dimensions des données
print(f'Train shape: {df_train.shape} - Test shape: {df_test.shape}')

# Préparation des données

In [ ]:
def preprocessing_content(df):
    # On commence par supprimer les stopwords
    stop_words = set(stopwords.words('english'))
    
    df['content'] = df['content'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))
    
    # On supprime la ponctuation
    df['content'] = df['content'].apply(lambda x: re.sub(r'[^\w\s]', '', x.lower()))    

    # On supprime les chiffres
    df['content'] = df['content'].apply(lambda x: re.sub(r'\d+', '', x))
    
    # On supprime les espaces en trop
    df['content'] = df['content'].apply(lambda x: x.strip())
    
    # On supprime les mots de moins de 3 lettres
    df['content'] = df['content'].apply(lambda x: ' '.join([word for word in x.split() if len(word) > 2]))
    
    # On applique le stemmer
    stemmer = PorterStemmer()
    
    # On applique le stemmer qui réduit les mots à leur racine
    df['content'] = df['content'].apply(lambda x: ' '.join([stemmer.stem(word) for word in x.split()]))
    
    return df

In [ ]:
# On applique la fonction de preprocessing
df_train = preprocessing_content(df_train)

In [ ]:
# On applique la fonction de preprocessing
df_test = preprocessing_content(df_test)

In [ ]:
# On sauvegarde le dataframe
df_train.to_csv('../data/train_preprocessed.csv', index=False)

In [ ]:
df_test.to_csv('../data/test_preprocessed.csv', index=False)

# Prédictions

In [ ]:
# On lis les données préprocessées
df_train = pd.read_csv('../data/train_preprocessed.csv')

In [ ]:
# On garde juste les colonnes content et label
df_train = df_train[['content', 'label']]

In [ ]:
# On convertie les labels en entiers
df_train['label'] = df_train['label'].astype('category').cat.codes

In [ ]:
# On affiche le haut du dataframe
df_train.head()

In [ ]:
#On prépare les données pour le modèle
X = df_train['content'].values
y = df_train['label'].values

In [ ]:
# On affiche la taille des données
print(f'X shape: {X.shape} - y shape: {y.shape}')

In [ ]:
# On affiche les classes
print(f'Classes: {np.unique(y)}')

In [ ]:
# On prépare le texte pour le modèle
vectorizer = CountVectorizer()

In [ ]:
# On transforme le texte en vecteur
X = vectorizer.fit_transform(X)

In [ ]:
# On affiche la taille des données
print(f'X shape: {X.shape} - y shape: {y.shape}')

In [ ]:
X_test = X[0:10000]
y_test = y[0:10000]

In [ ]:
X_valid = X[10000:20000]
y_valid = y[10000:20000]

# Création du modèle

In [ ]:
model = xgb.XGBClassifier()

In [ ]:
# On entraine le modèle
model.fit(X, y)

In [ ]:
# On affiche le score
print(f'Score: {model.score(X_valid, y_valid)}')

In [ ]:
# On affiche les prédictions
print(f'Predictions: {model.predict(X_test)}')

In [ ]:
# On affiche les probabilités
y_proba = model.predict_proba(X_test)

In [ ]:
y_proba

In [ ]:
# Conversion des probabilités en pourcentage
y_proba = np.round(y_proba * 100, 2)

In [ ]:
y_proba

In [ ]:
# On convertie les probabilités en dataframe
y_proba = pd.DataFrame(y_proba, columns=['proba_0', 'proba_1'])

In [ ]:
y_proba.iloc[0:10]

In [ ]:
def preprocessing_content(df):
    # On commence par supprimer les stopwords
    stop_words = set(stopwords.words('english'))
    
    df['content'] = df['content'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))
    
    # On supprime la ponctuation
    df['content'] = df['content'].apply(lambda x: re.sub(r'[^\w\s]', '', x.lower()))    

    # On supprime les chiffres
    df['content'] = df['content'].apply(lambda x: re.sub(r'\d+', '', x))
    
    # On supprime les espaces en trop
    df['content'] = df['content'].apply(lambda x: x.strip())
    
    # On supprime les mots de moins de 3 lettres
    df['content'] = df['content'].apply(lambda x: ' '.join([word for word in x.split() if len(word) > 2]))
    
    # On applique le stemmer
    stemmer = PorterStemmer()
    
    # On applique le stemmer qui réduit les mots à leur racine
    df['content'] = df['content'].apply(lambda x: ' '.join([stemmer.stem(word) for word in x.split()]))
    
    return df

In [ ]:
def preprocessing_https(df):
    df['https'] = df['https'].astype('category').cat.codes
    return df

In [ ]:
def preprocessing_tld(df):
    df['tld'] = df['tld'].astype('category').cat.codes
    return df